# Creating An Assembly

First create an empty coordinate system using bumblebee.CSYS

In [ ]:
import bumblebee as bb

In [ ]:
assm = bb.CSYS()
assm.plot()

## Bracket Assembly

Import the bracket and ball joint (from stl) as RigidBody objects and add them to the plot.

In [ ]:
bracket = bb.RigidBody(r"C:\Users\Elijah\Documents\PrinterAssembly\Stewart\Python\StewartBracket.stl", name="Bracket", units='cm')

In [ ]:
assm.add(bracket)

In [ ]:
ball = bb.RigidBody(r"C:\Users\Elijah\Documents\PrinterAssembly\Stewart\Python\Ball.stl", name="Ball", units='cm')

In [ ]:
assm.add(ball)

Currently these files are not in the correct location or orientation. Use the rotate/translate methods to move the ball joint to the correct location. \*2.54 modifiers are because the translations are in inches but the stl is in centimeters.

In [ ]:
ball.rotate_from_euler('y', 90, degrees=True)

In [ ]:
ball.rotate_from_euler('x', 90, degrees=True)

In [ ]:
ball.translate([-1*2.54,0,-1.875*2.54])

Now add a new Frame. We will place this frame at the center of the ball joint to use later for kinematic calculations.

In [ ]:
joint = bb.Frame()

In [ ]:
assm.add(joint)

In [ ]:
joint.translate([-1*2.54,0,-0.875*2.54])

This frame is not visible because it is covered entirely by the rigid body. To see the frame you can toggle the visiblity of the ball joint.

In [ ]:
ball.toggle_vis()

Retoggle the visibility so you can see the ball again.

In [ ]:
bracket.toggle_vis()

In order to copy, rotate, and translate this assembly as a group it would be advantageous to group each of the pieces into a single assembly object. This is possible through bumblebee's RigidCollection object. First init an empty assembly object and then add each of the pieces.

In [ ]:
bk_assm = bb.RigidCollection(name='BracketAssm1')

In [ ]:
assm.add(bk_assm)

In [ ]:
for body in [bracket, ball, joint]:
    bk_assm.add(body)
    assm.tree.remove_node(body)

In [ ]:
bk_assm.body_frame.toggle_vis()

Finally, we can save this bracket assembly so that we can open and reference it later.

In [ ]:
bk_assm.to_json('BracketAssm')

## Base Plate Assembly

For the base plate assembly we will create a new blank coordinate system.

In [ ]:
import bumblebee as bb

In [ ]:
assm = bb.CSYS()
assm.plot()

Now we will add the base plate to the drawing.

In [ ]:
bplate = bb.RigidBody(r"C:\Users\Elijah\Documents\PrinterAssembly\Stewart\Python\BottomPlate.stl", name="BottomPlate", units='cm')

In [ ]:
assm.add(bplate)

Apply rotations to move it to the desired position.

In [ ]:
bplate.rotate_from_euler('x', 90, degrees=True)

Now we can add the bracket assembly to this coordinate system.

In [ ]:
bracket = bb.RigidCollection.from_json('BracketAssm.json')

In [ ]:
assm.add(bracket)

Now move the bracket into position

In [ ]:
bracket.translate([0,0,2.125*2.54])

In [ ]:
bracket.rotate_from_euler('z', 90, degrees=True)

In [ ]:
bracket.translate([0,(-4.25-0.625)*2.54, 0])

In [ ]:
bracket.translate([1.5*2.54, 0, 0])

We would like to have 6 brackets in total. To create copies of components in place you can use the duplicate function. Here we duplicate the bracket and move it to a new position.

In [ ]:
bracket2 = bracket.duplicate(name='Bracket2')

In [ ]:
assm.add(bracket2)

In [ ]:
bracket2.translate([-3*2.54, 0, 0])

To get the remaining 4 brackets, we will duplicate in place and rotate the brackets about the origin.

In [ ]:
for i, body in enumerate([bracket, bracket2]):
    for j, rot in enumerate([120, 240]):
        newbody = body.duplicate(name=f'Bracket{2*i+j+3}')
        assm.add(newbody)
        newbody.rotate_from_euler('z', rot, degrees=True, sweep=True)

Because we will use the base plate in our kinematics, it would be nice to make it, it's own assembly

In [ ]:
bplate_assm = bb.RigidCollection(name='BottomPlateAssm')

In [ ]:
assm.add(bplate_assm)

In [ ]:
for body in assm.tree.nodes[1:-1]:
    bplate_assm.add(body)
    assm.tree.remove_node(body)

Save the assembly

In [ ]:
bplate_assm.to_json('BottomPlateAssm')

## Verify movement

In [ ]:
import bumblebee as bb
assm = bb.Application()
assm.grid

In [ ]:
bplate_assm = bb.RigidCollection.from_json('BottomPlateAssm.json')

In [ ]:
assm.csys.add(bplate_assm)

In [ ]:
bplate_assm.translate([0,0,-3])

In [ ]:
bplate_assm.rotate_from_euler('x', -30, degrees=True)